#Background
This notebook is meant to turn Raw data from Field Day Lab's game, Lakeland, into data capable of being analyzed using methods of Quantitative Ethnography such as Epistemic Network Analysis.

# Read in Everything



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from math import ceil
import numpy as np
pd.options.display.max_columns = 1000
from google.colab import files
import urllib.request
from zipfile import ZipFile
from io import BytesIO

In [ ]:
# !ls "/content/drive"
import os
ENA_folder = '/content/drive/My Drive/Lakeland_ENA/Data'
data_fpath = lambda fname: os.path.join(ENA_folder, fname)
ENA_path = "large_files/Copy_Raw_LAKELAND_2019Dec.csv"
Proc_path = "Lakeland60.csv"
# ENA_path = "/content/drive/My Drive/Lakeland_ENA/Raw_Lakeland_2019Dec.csv"

# Establish Data Frame


In [ ]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=11QBpa7wt5jSrTBnT8hlGJ5CjLZhscLRP" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=11QBpa7wt5jSrTBnT8hlGJ5CjLZhscLRP" -o "Raw_LAKELAND_2019Dec.csv"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   3578      0 --:--:-- --:--:-- --:--:--  3578
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 9899M    0 9899M    0     0  65.1M      0 --:--:--  0:02:31 --:--:-- 85.2M


In [ ]:
!ls -l Raw_LAKELAND_2019Dec.csv

-rw-r--r-- 1 root root 10380870807 Jun 19 15:08 Raw_LAKELAND_2019Dec.csv


In [ ]:
#Read in CSV (raw)
df_Raw = pd.read_csv("Raw_LAKELAND_2019Dec.csv", sep='\t')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Read in CSV (proc)
df_Proc = pd.read_csv(data_fpath(Proc_path), engine='python')

In [ ]:
df_Raw.shape

(7055701, 71)

In [ ]:
df_Proc.shape

(60, 478)

In [ ]:
#Filter Raw Dataframe
df = df_Raw[df_Raw['session_id'].isin(df_Proc['sessID'])]

In [ ]:
# Drop Columns
df = df.drop(columns=['app_id_fast', 'curr_selection_type', 'app_version', 'player_id', 'event','money','speed','achievements', 'event_data_simple', 'tiles', 'farmbits', 'items', 'num_checkpoints_completed','curr_selection_data', 'camera_center','gametime','timestamp', 'num_food_produced', 'num_poop_produced', 'camera_center', 'num_milk_produced', 'tile_states','tile_nutritions','blurb_history','client_time','level', '(none)', 'road_builds','(not currently implemented)','emote_history','client_time_ms', 'server_time', 'remote_addr','req_id'])


In [ ]:
#Data Amount
count_row = df.shape[0]
count_col = df.shape[1]
print(count_col,count_row)

38 34167


In [ ]:
# Get names of indexes for which column event_cutom is 0, 2, ...
indexNames = (df[df["event_custom"] == 0].index)

# Delete these row indexes from dataFrame
df = df.drop(indexNames)

In [ ]:
count_row = df.shape[0]
count_col = df.shape[1]
print(count_col,count_row)

38 32807


#Assign Metadata


In [ ]:
#This snippet is for testing the progress and convo columns and redoing them
#df = df.drop(axis=1, columns=['progress','convo'])

In [ ]:
#Progress Column
#Assign 1 at start for each session_ID, customevent == 0
#Assign 2 after fertilizer tutorial customevent == 2, event_category == 5
#Assign 3 after event custom==2, event_category == 7
#Assign 4 after event custom == 2, event_category == 20

currentSessionValueDict = { }
for index,row in df.iterrows():
  sessionId = row['session_id']
  if ((not sessionId in currentSessionValueDict) or
      (row['event_custom'] == 1)
  ):
    currentSessionValueDict[sessionId] = 1

  progress = currentSessionValueDict[sessionId]
  if ((row['event_custom'] == 2 and row['event_category'] == 'begin' and row['event_label'] == 'buy_fertilizer') or
      (row['event_custom'] == 2 and row['event_category'] == 'begin' and row['event_label'] == 'buy_livestock') or
      (row['event_custom'] == 2 and row['event_category'] == 'begin' and row['event_label'] == 'rain')
  ):
    progress = progress + 1
    currentSessionValueDict[sessionId] = progress;
  
  df.loc[index, 'progress'] = progress

df['progress'].unique()


array([1., 2., 3., 4.])

In [ ]:
#Conversation Column
#Sessions with same Session ID = Same conversation until "Continue" then 2 until continue/startgame then 3
#Session = 1 when startgame and no continue, increment any time start game w/ continue

currentSessionConvoDict = { }
for index,row in df.iterrows():
  sessionId = row['session_id']
  if ((not sessionId in currentSessionConvoDict) or
      (row['event_custom'] == 1 and row['continue'] == '0')
  ):
    currentSessionConvoDict[sessionId] = 1

  convo = currentSessionConvoDict[sessionId]
  if (row['event_custom'] == 1 and row['continue'] == '1'):
    convo = convo + 1
    currentSessionConvoDict[sessionId] = convo;
  
  df.loc[index, 'convo'] = convo 

df['convo'].unique()

array([1., 2., 3., 4.])

In [ ]:
#Games Played
# Starts at 1 and goes up by one for each event_custom=0 event.

currentSessionValueDict = { }
for index,row in df.iterrows():
  sessionId = row['session_id']
  if ((not sessionId in currentSessionValueDict)):
    currentSessionValueDict[sessionId] = 0

  value = currentSessionValueDict[sessionId]
  if (row['event_custom'] == 1 and row['continue'] == '0'):
    value = value + 1
    currentSessionValueDict[sessionId] = value;
  
  if(value < 1):
    value = 1

  df.loc[index, 'games_played'] = value 

df['games_played'].unique()

array([1., 2., 3., 4.])

In [ ]:
#Speaker Column
#Assign Game to events 

playr = [3,4,5,6,7,8,9,10,11,12,13,14,15,16,30]

def f(row):
    if row['event_custom'] in playr :
        val = 'player'
    else:
        val = 'game'
    return val


df['speaker'] = df.apply(f, axis=1)

# Convert Data - Events and Descriptors
0. gamestate - not used
2. startgame - X
3. checkpoint - Tutorial
4. selecttile
4. selectfarmbit
1. selectitem
1. selectbuy
1. buy
1. cancelbuy
1. roadbuilds
1. tileuseselect
1. itemuseselect
1. togglenutrition
1. toggleshop
1. toggleachievements
1. skiptutorial
1. speed
1. achievement
1. farmbitdeath
1. blurb (Not implemented)
1. click (Not implemented)
1. rainstopped
1. history
1. endgame
1. emote
1. farmfail
1. bloom
1. farmharvested
1. milkproduced
1. poopproduced
1. debug
1. newfarmbit


In [ ]:
#Event Number to Words
mapping = {
    0:'gamestate', 1:'startgame:You set out to form a new town called Lakeland. Your people love to play in the water. Grow your town without destroying their lakes.', 2:'checkpoint', 3:'selecttile', 4:'selectfarmbit', 5:'selectitem', 6:'selectbuy', 7:'buy', 8:'cancelbuy', 9:'roadbuilds', 10:'tileuseselect', 11:'itemuseselect', 12:'togglenutrition', 13:'toggleshop', 14:'toggleachievements', 15:'skiptutorial', 16:'speed', 17:'achievement', 18:'farmbitdeath', 19:'blurb', 20:'click', 21:'rainstopped', 22:'history', 23:'endgame', 24:'emote', 25:'farmfail', 26:'bloom', 27:'farmharvested', 28:'milkproduced', 29:'poopproduced', 30:'debug', 31:'newfarmbit'}
df = df.replace({'event_custom': mapping})

In [ ]:
#Menu Toggles To Meaning
mapping = {'True':'Player Opened', 'False':'Player Closed', '0':'Player Closed'}
df = df.replace({'achievements_open':mapping})
df= df.replace({'shop_open':mapping})

nutr_mapping = {'False': 'Player Turned Off', 'True':'Player Turned On'}
df = df.replace({'to_state':nutr_mapping})

In [ ]:
#Emojis to Words - 24
mapping_emoji = {'0':'null', '1':'Im hungry', '2':'i need food', '3':'i need a nap!', '4':'i want to play in the water', '5':'im so sad', '6':'Im sick Puke Emoji', '7':'Yumface', '8':'Im tired sleepy face', '9':'Im happy happy face', '10':'Ive got my floatie on flamingo swim emoji', '11':'Off to market sale'}
mapping_emoji
df = df.replace({'emote_enum': mapping_emoji})


In [ ]:
#Speed Data - 16
#cur_speed
mapping_speed = {'0':'null', '1':'pause', '2':'play x 1', '3':'fast x 4', '4':'fast x 16',}
mapping_speed
df = df.replace({'cur_speed': mapping_speed})
#Prev_speed
df = df.replace({'prev_speed': mapping_speed})
#Manual
mapping_manual = {'0':'null', '1':'Changed by Player'}
mapping_manual
df = df.replace({'manual': mapping_manual})

In [ ]:
#Achievement Data - 17
mapping_achievement = {'0':'exist get a visitor', '1':'group 3 workers', '2':'town a small community', '3':	'city 10 townmembers', '4':'farmer own a farm!', '5':'farmers get three farms', '6':	'farm town 5 farms!', '7': 'mega farm 10 farm industry', '8':	'paycheck $500', '9':	'thousandair $1000','10':	'stability $5000','11':	'riches $10000', '12':	'bloom algae destroys one tile','13':	'big bloom algae spreads to 3 tiles','14':	'huge bloom you have an algae problem','15':	'massive bloom a whole lake destroyed'}
df = df.replace({'achievement': mapping_achievement})

In [ ]:
#Buy
buy_type_to_enum = {"null": 0,
    "home": '1',
    "food": '2',
    "farm": '3',
    "fertilizer": '4',
    "livestock": '5',
    "skimmer": '6',
    "sign": '7',
    "road": '8'}
buy_enum_to_type = {v:k for k,v in buy_type_to_enum.items()}

df = df.replace({'buy': buy_enum_to_type})

In [ ]:
#Success
succ_desc={
    'True':"I can buy this.",
    'False' : "I can't buy this.",
    1: "I bought it!",
    2: "I can't put that there..."}

df = df.replace({'success':succ_desc})

In [ ]:
#prev_mark
mark_type_to_enum = {'null': 0,
    "use": 1,
    "sell": 2,
    "feed": 3}
mark_enum_to_type = {v:k for k,v in mark_type_to_enum.items()}

df=df.replace({'prev_mark':mark_enum_to_type})

In [ ]:
df.head(n=20)

,id,app_id,session_id,persistent_session_id,event_custom,raining,event_category,event_label,event_type,continue,language,audio,fullscreen,tile,marks,farmbit,item,mark,buy,cost,curr_money,success,buy_hovers,selected_buy,prev_mark,to_state,shop_open,achievements_open,cur_speed,prev_speed,manual,achievement,grave,camera_history,emote_enum,client_time.1,session_n,http_user_agent,progress,convo,games_played,speaker
421625,54326183,LAKELAND,19110018560604900,19100616575410184,startgame:You set out to form a new town calle...,None,None,None,None,0,english,1,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:56:29,0,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,game
421626,54326210,LAKELAND,19110018560604900,19100616575410184,checkpoint,None,begin,build_a_house,tutorial,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:56:41,1,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,game
421627,54326234,LAKELAND,19110018560604900,19100616575410184,selecttile,None,None,None,None,None,None,None,None,"[0, 12, 1, 1, 21, 17]","[1, 1, 1, 1]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:56:47,2,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,player
421628,54326236,LAKELAND,19110018560604900,19100616575410184,selectbuy,None,None,None,None,None,None,None,None,None,None,None,None,None,home,1000,1000,I can buy this.,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:56:48,3,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,player
421630,54326327,LAKELAND,19110018560604900,19100616575410184,buy,None,None,None,None,None,None,None,None,"[0, 12, 1, 1, 25, 26]",None,None,None,None,home,None,None,I can buy this.,"[[0, 12, 1, 1, 17, 26, 0, -11023], [0, 12, 1, ...",None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:56:59,5,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,player
421631,54326328,LAKELAND,19110018560604900,19100616575410184,selecttile,None,None,None,None,None,None,None,None,"[0, 12, 1, 8, 25, 26]","[1, 1, 1, 1]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:56:59,6,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,player
421632,54326329,LAKELAND,19110018560604900,19100616575410184,newfarmbit,None,None,None,None,None,None,None,None,None,None,"[25, 26, 17, 3, 1, 255, 255, 255, 102]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:57:05,7,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,game
421633,54326334,LAKELAND,19110018560604900,19100616575410184,selectfarmbit,None,None,None,None,None,None,None,None,None,None,"[25, 26, 17, 3, 1, 254, 254, 254, 101]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:57:05,8,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,player
421634,54326335,LAKELAND,19110018560604900,19100616575410184,selecttile,None,None,None,None,None,None,None,None,"[0, 12, 1, 8, 25, 26]","[1, 1, 1, 1]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-02 00:57:05,9,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,player
421635,54326336,LAKELAND,19110018560604900,19100616575410184,emote,None,None,None,None,None,None,None,None,None,None,"[28, 23, 17, 2, 1, 252, 250, 250, 97]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Ive got my floatie on flamingo swim emoji,2019-12-02 00:57:08,10,Mozilla/5.0 (X11; CrOS x86_64 12499.51.0) Appl...,1.0,1.0,1.0,game


#JSON Manipulation


*   Tile
*   Items
* Farmbits




In [ ]:
#Check variation of nutrition

import json

def stats(dataSequence):
    return {
        'max': dataSequence.max(),
        'min': dataSequence.min(),
        'median': dataSequence.median(),
        'mean': dataSequence.mean(),
        'mode': dataSequence.mode(),
        'std':dataSequence.std()
    }


# Gets all the data that is not 'None' in a column
def getActualColumnData(columnLabel):
    return df[df[columnLabel] != 'None'][columnLabel]

def json2obj(x):
    return json.loads(x) if (x!=None and x!='None') else None

# Creates a lambda for mapping a dataFrame column from one value to another
def applyMapping(mapping):
    return lambda x: mapping[x] if (x!=None and x!='None') else None

# Creates a lambda for mapping a dataFrame column from an array of values to a 
# value from specified position in the array
def applyArrayPos(position):
    return lambda x: x[position-1] if (isinstance(x,list) and len(x)>=position) else None


def getData(dataConfig):
    column = getActualColumnData(dataConfig['columnLabel'])
    if(dataConfig['arrayPosition'] != None):
        column = column.map(json2obj).map(applyArrayPos(dataConfig['arrayPosition']))
    if(dataConfig['mapping'] != None):
        column = column.map(applyMapping(dataConfig['mapping']))
    return column

nutritionConfig = {
    'name' : 'nutrition',
    'columnLabel': 'tile',
    'arrayPosition': None,
    'mapping': None,
}

nutritionData = getData(nutritionConfig)

nutritionData

421627     [0, 12, 1, 1, 21, 17]
421630     [0, 12, 1, 1, 25, 26]
421631     [0, 12, 1, 8, 25, 26]
421634     [0, 12, 1, 8, 25, 26]
421641     [0, 12, 1, 1, 22, 24]
                   ...          
7034311    [0, 63, 6, 6, 25, 20]
7034312    [10, 0, 1, 9, 24, 21]
7034317     [4, 0, 1, 9, 28, 22]
7034322    [0, 12, 1, 1, 22, 26]
7034323    [0, 12, 1, 1, 23, 23]
Name: tile, Length: 8260, dtype: object

In [ ]:
#Tile Type Change

#tile_type to enum conversion
tile_type_to_enum = {
    "null": 0,
    "land": 1,
    "rock": 2,
    "grave": 3,
    "sign": 4,
    "lake": 5,
    "shore": 6,
    "forest": 7,
    "home": 8,
    "farm": 9,
    "livestock": 10,
    "road": 11}
tile_enum_to_type = {v:k for k,v in tile_type_to_enum.items()}

#t= df["tile"]

import json
#t_arr = json.loads('t')

def nutritionToDescription(nutritionValue):
    if nutritionValue < 3:
        return "low"
    elif nutritionValue < 24:
        return "medium"
    else: 
        return "high"

def progressToDescription(progressValue):
    if progressValue < 64:
        return "starting"
    elif nutritionValue < 191:
        return "in progress"
    else: 
        return "almost done"

#outputs: progress at [0], high/med/low nutrition, was og_type now my_type, position at x,y
def tile_myfunc(tile_json):
  if(tile_json == 'None'):
      return None
  
  tile_array = json.loads(tile_json)
  return convertTileArray(tile_array)

def convertTileArray(tile_array):
  val255 = tile_array[0]
  nutrition255 = tile_array[1]
  og_type, my_type = tile_array[2], tile_array[3]
  tx,ty = tile_array[4],tile_array[5]

  progress = "progress at {}".format(progressToDescription(val255))
  curr_nutrition = "{} nutrition".format(nutritionToDescription(nutrition255))
  history = "was {} and now {}".format(tile_enum_to_type[og_type], tile_enum_to_type[my_type])
  position = "position at ({},{})".format(tx,ty)
  
  return progress, curr_nutrition, history, position


df['tile'] = df['tile'].map(tile_myfunc)


In [ ]:
#Farmbit Change

def valueToDescription(Value):
    if Value < 64:
        return "not"
    elif Value < 191:
        return "somewhat"
    else: 
        return "very"

#Names
whois_enum_to_name = {
    0: 'Peter',	
    1: 'Paul',
    2: 'Mary',
    3: 'John',
    4: 'George',
    5:	'Ringo'	,
    6:	'Yoko'	,
    7:	'Stevie',	
    8:	'Saanvi',	
    9:	'Nethra',	
    10:	'Meha',	
    11:	'Sidney',	
    12:	'Lucy',
    13:	'Belden',	
    14:	'Henry'	,
    15:	'Alejandro',	
    16:	'Victor',	
    17:'Richard'}

#fb state
fb_state_to_enum = {
    "null": '0',
    "dire": '1',
    "desperate": '2',
    "motivated": '3'}
fb_enum_to_state = {v:k for k,v in fb_state_to_enum.items()}


#Job_State to enum conversion
job_state_to_enum = {
    "null": 0,
    "get": 1,
    "seek": 2,
    "act": 3,}
job_enum_to_state = {v:k for k,v in job_state_to_enum.items()}

#enum to Job_Type
job_type_to_enum = {'null':0, 'idle':1, "wait":2, "eat":3, "sleep":4, 'play':5, 'plant' : 6, 'harvest':7, 'feed':8, 'fertilize':9, 'milk':10, 'export':11}
job_enum_to_type = {v:k for k,v in job_type_to_enum.items()}


import json

#outputs: who, where, job, personal stats
def fb_myfunc(fb_json):
  if(fb_json == 'None'):
      return None
 
  fb_array = json.loads(fb_json)
  tx,ty = fb_array[0],fb_array[1]
  name = fb_array[2]
  job_state, job_type = fb_array[3], fb_array[4]
  fullness,energy,joy,fulfillment = fb_array[5],fb_array[6],fb_array[7], fb_array[8]
  
  who = "{}".format(whois_enum_to_name[name]) 
  position = "at ({},{})".format(tx,ty)
  job = "I'm {} to {}.".format(job_enum_to_state[job_state], job_enum_to_type[job_type])
  state = "I'm feeling {} full {} energetic {} joyful {} fulfilled.".format(valueToDescription(fullness),valueToDescription(energy),valueToDescription(joy),valueToDescription(fulfillment))
  
  return who,position,job,state


df['farmbit']=df['farmbit'].map(fb_myfunc)

In [ ]:
df.tail(n=20)

,id,app_id,session_id,persistent_session_id,event_custom,raining,event_category,event_label,event_type,continue,language,audio,fullscreen,tile,marks,farmbit,item,mark,buy,cost,curr_money,success,buy_hovers,selected_buy,prev_mark,to_state,shop_open,achievements_open,cur_speed,prev_speed,manual,achievement,grave,camera_history,emote_enum,client_time.1,session_n,http_user_agent,progress,convo,games_played,speaker
7034310,60285633,LAKELAND,19110620573509880,19110620545193596,selectitem,None,None,None,None,None,None,None,None,None,None,None,"[25, 20, 4, 1]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-08 02:05:18,403,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,player
7034311,60285636,LAKELAND,19110620573509880,19110620545193596,selecttile,None,None,None,None,None,None,None,None,"(progress at starting, high nutrition, was sho...","[1, 1, 1, 1]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-08 02:05:18,404,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,player
7034312,60285637,LAKELAND,19110620573509880,19110620545193596,selecttile,None,None,None,None,None,None,None,None,"(progress at starting, low nutrition, was land...","[1, 1, 1, 1]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-08 02:05:21,405,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,player
7034313,60285638,LAKELAND,19110620573509880,19110620545193596,emote,None,None,None,None,None,None,None,None,None,None,"(Belden, at (28,21), I'm seek to idle., I'm fe...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,i want to play in the water,2019-12-08 02:05:21,406,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,game
7034314,60285639,LAKELAND,19110620573509880,19110620545193596,emote,None,None,None,None,None,None,None,None,None,None,"(Belden, at (28,20), I'm act to play., I'm fee...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Im happy happy face,2019-12-08 02:05:21,407,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,game
7034315,60285640,LAKELAND,19110620573509880,19110620545193596,emote,None,None,None,None,None,None,None,None,None,None,"(Belden, at (28,20), I'm act to idle., I'm fee...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Ive got my floatie on flamingo swim emoji,2019-12-08 02:05:21,408,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,game
7034316,60285641,LAKELAND,19110620573509880,19110620545193596,rainstopped,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-08 02:05:22,409,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,game
7034317,60285642,LAKELAND,19110620573509880,19110620545193596,selecttile,None,None,None,None,None,None,None,None,"(progress at starting, low nutrition, was land...","[1, 1, 1, 1]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2019-12-08 02:05:22,410,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,player
7034318,60285646,LAKELAND,19110620573509880,19110620545193596,emote,None,None,None,None,None,None,None,None,None,None,"(Belden, at (23,21), I'm act to idle., I'm fee...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Im hungry,2019-12-08 02:05:24,411,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,4.0,2.0,1.0,game
7034319,60285649,LAKELAND,19110620573509880,19110620545193596,emote,None,None,None,None,None,None,None,None,None,None,"(Belden, at (27,18), I'm seek to idle., I'm fe...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Ive got my floatie on flamingo swim emoji,2019-12-08 02:05:2

In [ ]:
#Items

#Item_Type to enum conversion
item_type_to_enum = {
    "null": 0,
    "water": 1,
    "food": 2,
    "poop": 3,
    "fertilizer": 4,
    "milk": 5}  
item_enum_to_type = {v:k for k,v in item_type_to_enum.items()}

#Item_Marks to Enum
mark_type_to_enum = {'null': 0,
    "use": 1,
    "sell": 2,
    "feed": 3}
mark_enum_to_type = {v:k for k,v in mark_type_to_enum.items()}


import json

#outputs: where, what, how
def item_myfunc(item_json):
  if(item_json == 'None'):
      return None

  item_array = json.loads(item_json)
  tx,ty = item_array[0],item_array[1]
  item, mark = item_array[2], item_array[3]
  
  where = "at ({},{})".format(tx,ty) 
  what = "{} for {}".format((item_enum_to_type[item]),(mark_enum_to_type[mark]))
  
  return where,what

df['item']= df['item'].map(item_myfunc)  

In [ ]:
#Marks

#Item_Marks to Enum
mark_type_to_enum = {'null': 0,
    "use": 1,
    "sell": 2,
    "feed": 3}

mark_enum_to_type = {v:k for k,v in mark_type_to_enum.items()}

#Marks outputs: item marks for item, tile, etc.
def mark_myfunc(mark_json):
  if(mark_json == 'None'):
      return None

  mark_array = json.loads(mark_json)
  mark_1, mark_2 = mark_array[0],mark_array[1]
  mark_unused, mark_unused2 = mark_array[2], mark_array[3]

  desc_mark="Items marked for {} and {}.".format((mark_enum_to_type[mark_1]),(mark_enum_to_type[mark_2]))

  return desc_mark


df.marks = df['marks'].map(mark_myfunc)

In [ ]:
#Hovers

def buyHovers_func(json_str):
    if json_str=='None':
      return None
    
    buyHovers_arr = json.loads(json_str)
    return convertBuyHoversArray(buyHovers_arr)

def convertBuyHoversArray(buyHovers_arr):
    buyHoversDescriptionArray = []
    for tile_arr in buyHovers_arr:
        fullTileDescription = convertTileArrayForBuyHovers(tile_arr);
        buyHoversDescriptionArray.append(fullTileDescription)
    return buyHoversDescriptionArray



def convertTileArrayForBuyHovers(tile_array):
  val255 = tile_array[0]
  nutrition255 = tile_array[1]
  og_type, my_type = tile_array[2], tile_array[3]
  tx,ty = tile_array[4],tile_array[5]

  progress = "progress at {}".format(progressToDescription(val255))
  curr_nutrition = "{} nutrition".format(nutritionToDescription(nutrition255))
  history = "was {} and now {}".format(tile_enum_to_type[og_type], tile_enum_to_type[my_type])
  position = "position at ({},{})".format(tx,ty)
  
  return progress, curr_nutrition, history, position

df.buy_hovers = df.buy_hovers.map(buyHovers_func)

#Combine Descriptor Columns


In [ ]:
#Text Data Column

#For every description, add column name + description

if ('text_data' not in df):
  df['text_data'] = ''

columnsToCombine = ['event_category', 'event_label', 'continue', 'tile',
       'marks', 'farmbit', 'item', 'buy', 'cost', 'curr_money',
       'success', 'buy_hovers', 'selected_buy', 'prev_mark',
       'to_state', 'shop_open', 'achievements_open', 'cur_speed',
       'prev_speed', 'manual', 'achievement', 'emote_enum'];

for index, row in df.iterrows():
  if(df.loc[index, 'text_data'] == ''):
    textValue = ''
    for columnName in columnsToCombine:
      columnValue = df.loc[index, columnName]
      if columnValue != 'None' and columnValue != None:
        if textValue != '':
          textValue = textValue + ', '
        textValue = textValue + "{}: {}".format(columnName, columnValue)


    textValue = '{event_custom: ' + df.loc[index, 'event_custom'] + ', data:{' + textValue + '}}'
    
    df.loc[index, 'text_data'] = textValue



In [ ]:
df['text_data']



421625     {event_custom: startgame:You set out to form a...
421626     {event_custom: checkpoint, data:{event_categor...
421627     {event_custom: selecttile, data:{tile: ('progr...
421628     {event_custom: selectbuy, data:{buy: home, cos...
421630     {event_custom: buy, data:{tile: ('progress at ...
                                 ...                        
7034325    {event_custom: selectfarmbit, data:{farmbit: (...
7034326    {event_custom: farmbitdeath, data:{farmbit: ('...
7034327    {event_custom: checkpoint, data:{event_categor...
7034328    {event_custom: speed, data:{cur_speed: play x ...
7034329    {event_custom: checkpoint, data:{event_categor...
Name: text_data, Length: 32807, dtype: object

#Add Meta Data

1.   Add Conversation Markers - Continue = 2, etc
2.   Add Game vs. Player label



#Save To CSV

In [ ]:
ENA_folder = '/content/drive/My Drive/Lakeland_ENA'
data_fpath2 = lambda fname: os.path.join(ENA_folder, fname)
ENA_path2 = data_fpath2("Hopes.And.Dreams.csv")
# ENA_path2 = "/content/drive/My Drive/Lakeland_ENA/CuratedData.csv"
df.to_csv(ENA_path2, sep=',', index = None)

In [ ]:
df.describe

<bound method NDFrame.describe of                id    app_id         session_id  persistent_session_id  \
421625   54326183  LAKELAND  19110018560604900      19100616575410184   
421626   54326210  LAKELAND  19110018560604900      19100616575410184   
421627   54326234  LAKELAND  19110018560604900      19100616575410184   
421628   54326236  LAKELAND  19110018560604900      19100616575410184   
421630   54326327  LAKELAND  19110018560604900      19100616575410184   
...           ...       ...                ...                    ...   
7034325  60285664  LAKELAND  19110620573509880      19110620545193596   
7034326  60285665  LAKELAND  19110620573509880      19110620545193596   
7034327  60285666  LAKELAND  19110620573509880      19110620545193596   
7034328  60285667  LAKELAND  19110620573509880      19110620545193596   
7034329  60285682  LAKELAND  19110620573509880      19110620545193596   

                                              event_custom raining  \
421625   startgame: